# Visualize and analyze the geospatial features of protests/demonstrations using the geoprotest API hosted on Rapid API

* You need a valid Rapid API account
* The system environment must contain your Rapid API key ('x_rapidapi_key') and is send as a http header parameter.

In [12]:
from arcgis import GIS
from arcgis.features import FeatureSet
from datetime import datetime, timedelta
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from georapid.formats import OutFormat
from georapid.protests import aggregate, articles, hotspots

In [8]:
help(EnvironmentClientFactory.create_client_with_host)

Help on function create_client_with_host in module georapid.factory:

create_client_with_host(host)
    Creates a new client using 'x_rapidapi_key' enironment variable.
    Raises a ValueError when this variable is not defined!



In [9]:
host = "geoprotests.p.rapidapi.com"
client: GeoRapidClient = EnvironmentClientFactory.create_client_with_host(host)

In [10]:
help(aggregate)

Help on function aggregate in module georapid.protests:

aggregate(client: georapid.client.GeoRapidClient, date: datetime.datetime = None, format=<OutFormat.GEOJSON: 1>)
    Aggregates the broadcasted news related to protests/demonstrations using a spatial grid and returns the features as hexagonal bins.
    The date is optional. When not specified, we return the features of the last 24 hours.
    The underlying hosted feature service saves the last 90 days and yesterday should be the latest available date.
    The format can be GeoJSON or Esri JSON.



## Connect to ArcGIS Online anonymously

In [5]:
gis = GIS()

## Use case: Visualize the news related to protests of 24th February 2022

In [13]:
date_of_interest = datetime(2022, 2, 24)

world_map = gis.map()
world_map.basemap = 'dark-gray-vector'

protests_featureset = FeatureSet.from_dict(aggregate(client, date_of_interest, OutFormat.ESRI))
if protests_featureset.sdf.empty:
    print("The feature set is empty!")
else:
    protests_featureset.sdf.spatial.plot(world_map,
                                     renderer_type='c', # for class breaks renderer
                                     method='esriClassifyNaturalBreaks', # classification algorithm
                                     class_count=5, # choose the number of classes
                                     col='count', # numeric column to classify
                                     cmap='YlOrRd', # color map to pick colors from for each class
                                     alpha=0.35 # specify opacity
    )
world_map

MapView(layout=Layout(height='400px', width='100%'))

## Use Case: Visualize the hotspots related to protests of 24th February 2022

In [14]:
world_map = gis.map()
world_map.basemap = 'dark-gray-vector'

protests_featureset = FeatureSet.from_dict(hotspots(client, date_of_interest, OutFormat.ESRI))
if protests_featureset.sdf.empty:
    print("The feature set is empty!")
else:
    protests_featureset.sdf.spatial.plot(world_map,
                                     renderer_type='c', # for class breaks renderer
                                     method='esriClassifyNaturalBreaks', # classification algorithm
                                     class_count=5, # choose the number of classes
                                     col='count', # numeric column to classify
                                     cmap='YlOrRd', # color map to pick colors from for each class
                                     alpha=0.35 # specify opacity
    )
world_map

MapView(layout=Layout(height='400px', width='100%'))

In [15]:
protests_featureset.sdf

,OBJECTID,name,timestamp,count,SHAPE
0,11457,"Moscow, Moskva, Russia",2022-02-24,544,"{""x"": 37.6156, ""y"": 55.7522, ""spatialReference..."
1,11458,"Ottawa, Ontario, Canada",2022-02-24,172,"{""x"": -75.7, ""y"": 45.4167, ""spatialReference"":..."
2,11459,"Saint Petersburg, Sankt-Peterburg, Russia",2022-02-24,161,"{""x"": 30.2642, ""y"": 59.8944, ""spatialReference..."
3,11460,"Berlin, Berlin, Germany",2022-02-24,125,"{""x"": 13.4, ""y"": 52.5167, ""spatialReference"": ..."
4,11461,"Washington, Washington, United States",2022-02-24,75,"{""x"": -77.0364, ""y"": 38.8951, ""spatialReferenc..."
5,11462,"London, London, City Of, United Kingdom",2022-02-24,61,"{""x"": -0.1167, ""y"": 51.5, ""spatialReference"": ..."
6,11463,"Warsaw, (Pl67), Poland",2022-02-24,57,"{""x"": 21, ""y"": 52.25, ""spatialReference"": {""wk..."
